# Code For Grain Encrption Cipher - A Stream Cipher

In [1]:
import numpy as np
import binascii

In [2]:
## Hepler functions for interconversion of bits and strings

def text_to_bits(text, encoding='utf-8', errors='surrogatepass'):
    bits = bin(int(binascii.hexlify(text.encode(encoding, errors)), 16))[2:]
    return bits.zfill(8 * ((len(bits) + 7) // 8))

def text_from_bits(bits, encoding='utf-8', errors='surrogatepass'):
    n = int(bits, 2)
    return int2bytes(n).decode(encoding, errors)

def int2bytes(i):
    hex_string = '%x' % i
    n = len(hex_string)
    return binascii.unhexlify(hex_string.zfill(n + (n & 1)))

def string2bits(s=''):
    return [bin(ord(x))[2:].zfill(8) for x in s]

def bits2string(b=None):
    return ''.join([chr(int(x, 2)) for x in b])

In [3]:
## Main building blocks of Grain Cipher, namely a Linear Feedback Shift Register (LFSR)
## and a Non-linear Feedback Shift Register (NFSR)

lfsr = np.zeros(80,dtype=bool)
nfsr = np.zeros(80,dtype=bool)

## Initialise LFSR and NFSR using IV and Secret Key
## First load the NFSR with the key bits, bi = ki, 0 ≤ i ≤ 79,
## then load the first 64 bits of the LFSR with the IV, si = IVi, 0 ≤ i ≤ 63.
## The remaining bits of the LFSR are filled with ones, si = 1, 64 ≤ i ≤ 79.
## Because of this the LFSR cannot be initialized to the all zero state.

def init(iv,key):
    iv_bin = string2bits(iv)
    iv_bin = ''.join(iv_bin)
    lfsr[:64] = [bool(int(iv_bin[ix])) for ix in range(len(iv_bin))]
    lfsr[64:] = 1
    key_bin = string2bits(key)
    key_bin = ''.join(key_bin)
    nfsr[:] = [bool(int(key_bin[ix])) for ix in range(len(key_bin))]

    
## The cipher is clocked 160 times without producing any running key
## The output of the filter function, h(x), is fed back and xored with the input, both to the LFSR and to the NFSR

def clock():
    hx=0
    fx=0
    gx=0
    global lfsr
    global nfsr
    for ix in range(160):
        fx = lfsr[62] ^ lfsr[51] ^ lfsr[38] ^ lfsr[23] ^ lfsr[13] ^ lfsr[0] ^ hx
        gx = hx ^ nfsr[0] ^ nfsr[63] ^ nfsr[60] ^ nfsr[52] ^ nfsr[45] ^ nfsr[37] ^ nfsr[33] ^ nfsr[28] ^ nfsr[21] ^ nfsr[15] ^ nfsr[19] ^ nfsr[0] ^ nfsr[63] & nfsr[60] ^ nfsr[37] & nfsr[33] ^ nfsr[15] & nfsr[9] ^ nfsr[60] & nfsr[52] & nfsr[45] ^ nfsr[33] & nfsr[28] & nfsr[21] ^ nfsr[63] & nfsr[45] & nfsr[28] & nfsr[9] ^ nfsr[60] & nfsr[52] & nfsr[37] & nfsr[33] ^ nfsr[63] & nfsr[60] & nfsr[21] & nfsr[15] ^ nfsr[63] & nfsr[60] & nfsr[52] & nfsr[45] & nfsr[37] ^ nfsr[33] & nfsr[28] & nfsr[21] & nfsr[15] & nfsr[9] ^ nfsr[52] & nfsr[45] & nfsr[37] & nfsr[33] & nfsr[28] & nfsr[21]
        x0 = lfsr[0]
        x1 = lfsr[25]
        x2 = lfsr[46]
        x3 = lfsr[64]
        x4 = nfsr[63]
        hx = x1 ^ x4 ^ x0 & x3 ^ x2 & x3 ^ x3 & x3 ^ x0 & x1 & x2 ^ x0 & x2 & x3 ^ x0 & x2 & x4 ^ x1 & x2 & x4 ^ x2 & x3 & x4
        lfsr[:-1] = lfsr[1:]
        lfsr[-1] = fx
        nfsr[:-1] = nfsr[1:]
        nfsr[-1] = gx    
        
        
        
## Return a stream generator which implements the filter function

def gen_key_stream():
    hx = 0
    while True:
        fx = lfsr[62] ^ lfsr[51] ^ lfsr[38] ^ lfsr[23] ^ lfsr[13] ^ lfsr[0]
        gx = nfsr[0] ^ nfsr[63] ^ nfsr[60] ^ nfsr[52] ^ nfsr[45] ^ nfsr[37] ^ nfsr[33] ^ nfsr[28] ^ nfsr[21] ^ nfsr[15] ^ nfsr[19] ^ nfsr[0] ^ nfsr[63] & nfsr[60] ^ nfsr[37] & nfsr[33] ^ nfsr[15] & nfsr[9] ^ nfsr[60] & nfsr[52] & nfsr[45] ^ nfsr[33] & nfsr[28] & nfsr[21] ^ nfsr[63] & nfsr[45] & nfsr[28] & nfsr[9] ^ nfsr[60] & nfsr[52] & nfsr[37] & nfsr[33] ^ nfsr[63] & nfsr[60] & nfsr[21] & nfsr[15] ^ nfsr[63] & nfsr[60] & nfsr[52] & nfsr[45] & nfsr[37] ^ nfsr[33] & nfsr[28] & nfsr[21] & nfsr[15] & nfsr[9] ^ nfsr[52] & nfsr[45] & nfsr[37] & nfsr[33] & nfsr[28] & nfsr[21]
        x0 = lfsr[0]
        x1 = lfsr[25]
        x2 = lfsr[46]
        x3 = lfsr[64]
        x4 = nfsr[63]
        hx = x1 ^ x4 ^ x0 & x3 ^ x2 & x3 ^ x3 & x3 ^ x0 & x1 & x2 ^ x0 & x2 & x3 ^ x0 & x2 & x4 ^ x1 & x2 & x4 ^ x2 & x3 & x4
        lfsr[:-1] = lfsr[1:]
        lfsr[-1] = fx
        nfsr[:-1] = nfsr[1:]
        nfsr[-1] = gx
        yield hx
        
        
        
def encrypt(iv,key,plain, resume=False):
    
    if not resume :
        init(iv,key)
        clock()
    
    
    plain = text_to_bits(plain)
    stream = gen_key_stream()
    
    def printNext(stream) :
        a = next(stream)
        #print(a)
        return a
    
    cipher = [str(int(bool(int(plain[ix]))^printNext(stream))) for ix in range(len(plain))]
    cipher = ''.join(cipher)
    return cipher



def decrypt(iv,key,cipher):
    init(iv,key)
    clock()
    stream = gen_key_stream()
    plain = [str(int(bool(int(cipher[ix]))^next(stream))) for ix in range(len(cipher))]
    plain = ''.join(plain)
    plain = text_from_bits(plain)
    return plain



# Code for Compression

In [7]:
def decimalToBinary(num , places):
    return ('{0:0' +  str(places) + 'b}').format(num)


def binaryToDecimal(binary):
    return int(binary, 2)
        
def xorBinary(binary,key = "00000000"):
    i=0
    encBin = ""
    while(i<8):
        if(binary[i] == '0' and key[i] == '0') or (binary[i] == '1' and key[i] == '1'):
            encBin+="0"
        else:
            encBin+="1"
        i+=1
        
#     print(binary , key , encBin)    
    return encBin
    
def compressBinary(binary):
    i=0
    compressBin = ""
    while(i<8):
        if(i%2 == 0):
            if((binary[i] == '0' and binary[i+1] == '0') or (binary[i] == '1' and binary[i+1] == '1')):
                compressBin+="0"
            else:
                compressBin+="1"
        i+=1
        
    return compressBin

def to_twoscomplement(bits, value):
    if value < 0:
        value = ( 1<<bits ) + value
    formatstring = '{:0%ib}' % bits
    return formatstring.format(value)


# Code For Delta De-Coding

In [3]:
import sys, traceback
import time , socket

In [59]:
def server_program():
    # get the hostname
    host = '192.168.43.136'
    port = 5000  # initiate port no above 1024
    count = 0
    last = None
    org_data = []
    start_time = None
    end_time = None

    
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)  # get instance
    server_socket.bind((host, port))  # bind host address and port together
    server_socket.listen(1)

    try :
        connection,client_address = server_socket.accept()
        while True:
            data = connection.recv(1)
            if not data:
                end_time = int(time.time() * 1000)
                print("Time in MS :" , end_time - start_time)
                print("Packet Transfered :" , count)
                connection.close()
                server_socket.close()
                return count , (end_time - start_time)
            
            data = int.from_bytes(data, byteorder='little')
            mask = 15 # 0000 1111            
            count +=1
                   
            if last == None :
                start_time = int(time.time() * 1000)
                last = data
                org_data.append(last)
            else :
                first_byte = (data >> 4) & mask
                if(first_byte > 7) :
                    first_byte -= 16
                
                second_byte = (data) & mask
                if(second_byte>7) :
                    second_byte -= 16
                    
                last += first_byte
                org_data.append(last)
                last += second_byte
                org_data.append(last)
                
                
            
    except Exception:
        print("Some Error Occured")
        traceback.print_exc()
        connection.close()
        server_socket.close()
        

In [49]:
cdata = server_program()

Time in MS : 1093
Packet Transfered : 464496


In [62]:
di = {}

for i in range(125):
    packets, ttime = server_program()
    if(di.get(packets)) :
        di[packets].append(ttime)
    else :
        di[packets] = [ttime]
    time.sleep(3)

Time in MS : 1228
Packet Transfered : 464496
Time in MS : 999
Packet Transfered : 464496
Time in MS : 1062
Packet Transfered : 464496
Time in MS : 1218
Packet Transfered : 464496
Time in MS : 1031
Packet Transfered : 464496
Time in MS : 1156
Packet Transfered : 464496
Time in MS : 1282
Packet Transfered : 464496
Time in MS : 1015
Packet Transfered : 464496
Time in MS : 1021
Packet Transfered : 464496
Time in MS : 1311
Packet Transfered : 464496
Time in MS : 1296
Packet Transfered : 464496
Time in MS : 1077
Packet Transfered : 464496
Time in MS : 1015
Packet Transfered : 464496
Time in MS : 984
Packet Transfered : 464496
Time in MS : 986
Packet Transfered : 464496
Time in MS : 1140
Packet Transfered : 464496
Time in MS : 1124
Packet Transfered : 464496
Time in MS : 1172
Packet Transfered : 464496
Time in MS : 984
Packet Transfered : 464496
Time in MS : 1036
Packet Transfered : 464496
Time in MS : 1093
Packet Transfered : 464496
Time in MS : 1068
Packet Transfered : 464496
Time in MS : 9

In [64]:
print(di)

{464496: [1228, 999, 1062, 1218, 1031, 1156, 1282, 1015, 1021, 1311, 1296, 1077, 1015, 984, 986, 1140, 1124, 1172, 984, 1036, 1093, 1068, 960, 1046, 1140], 232248: [577, 531, 625, 561, 609, 546, 499, 483, 531, 484, 500, 546, 562, 484, 484, 486, 468, 515, 484, 484, 452, 514, 578, 571, 515], 116124: [265, 344, 358, 265, 281, 327, 265, 265, 289, 249, 468, 312, 312, 265, 280, 233, 266, 249, 265, 312, 250, 296, 249, 249, 233], 58062: [124, 140, 124, 124, 125, 125, 109, 140, 125, 110, 139, 140, 140, 109, 171, 156, 124, 140, 110, 125, 125, 109, 140, 125, 140], 29031: [77, 78, 62, 62, 94, 77, 68, 78, 62, 78, 70, 93, 84, 89, 62, 62, 77, 129, 63, 74, 68, 61, 62, 71, 62]}


In [68]:
import json
print(json.dumps(di, indent=4))

{
    "464496": [
        1228,
        999,
        1062,
        1218,
        1031,
        1156,
        1282,
        1015,
        1021,
        1311,
        1296,
        1077,
        1015,
        984,
        986,
        1140,
        1124,
        1172,
        984,
        1036,
        1093,
        1068,
        960,
        1046,
        1140
    ],
    "232248": [
        577,
        531,
        625,
        561,
        609,
        546,
        499,
        483,
        531,
        484,
        500,
        546,
        562,
        484,
        484,
        486,
        468,
        515,
        484,
        484,
        452,
        514,
        578,
        571,
        515
    ],
    "116124": [
        265,
        344,
        358,
        265,
        281,
        327,
        265,
        265,
        289,
        249,
        468,
        312,
        312,
        265,
        280,
        233,
        266,
        249,
        265,
        312,
        

# Code for Un-Compressed

In [6]:


def server_program_uc():
    # get the hostname
    host = '192.168.43.136'
    port = 5000  # initiate port number above 1024
    count = 0
    org_data = []
    start_time = None
    end_time = None

    
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)  # get instance
    server_socket.bind((host, port))  # bind host address and port together
    server_socket.listen(1)

    try :
        connection,client_address = server_socket.accept()
        while True:
            data = connection.recv(1)
            if not data:
                # print(start_time)
                end_time = int(time.time() * 1000)
                print("Time in MS :" , end_time - start_time)
                print("Packet Transfered :" , count)
                connection.close()
                server_socket.close()
                return count , (end_time - start_time)
            
            data = int.from_bytes(data, byteorder='little')
                               
            if start_time == None :
                # print(start_time)
                start_time = int(time.time() * 1000)
            
            org_data.append(data)
            count += 1
                
            
    except Exception:
        print("Some Error Occured")
        traceback.print_exc()
        connection.close()
        server_socket.close()
        

In [45]:
ucdata = server_program_uc()

Time in MS : 2405
Packet Transfered : 928991


In [8]:
di_uc = {}

for i in range(125):
    packets, ttime = server_program_uc()
    if(di_uc.get(packets)) :
        di_uc[packets].append(ttime)
    else :
        di_uc[packets] = [ttime]
    time.sleep(3)

Time in MS : 1895
Packet Transfered : 928991
Time in MS : 1936
Packet Transfered : 928991
Time in MS : 2077
Packet Transfered : 928991
Time in MS : 1952
Packet Transfered : 928991
Time in MS : 1890
Packet Transfered : 928991
Time in MS : 2233
Packet Transfered : 928991
Time in MS : 2202
Packet Transfered : 928991
Time in MS : 2483
Packet Transfered : 928991
Time in MS : 1983
Packet Transfered : 928991
Time in MS : 1858
Packet Transfered : 928991
Time in MS : 2111
Packet Transfered : 928991
Time in MS : 2108
Packet Transfered : 928991
Time in MS : 2093
Packet Transfered : 928991
Time in MS : 1905
Packet Transfered : 928991
Time in MS : 2452
Packet Transfered : 928991
Time in MS : 2314
Packet Transfered : 928991
Time in MS : 2124
Packet Transfered : 928991
Time in MS : 2681
Packet Transfered : 928991
Time in MS : 2342
Packet Transfered : 928991
Time in MS : 2139
Packet Transfered : 928991
Time in MS : 2132
Packet Transfered : 928991
Time in MS : 2718
Packet Transfered : 928991
Time in MS

In [11]:
import json
print(json.dumps(di_uc , indent=2))

{
  "928991": [
    1895,
    1936,
    2077,
    1952,
    1890,
    2233,
    2202,
    2483,
    1983,
    1858,
    2111,
    2108,
    2093,
    1905,
    2452,
    2314,
    2124,
    2681,
    2342,
    2139,
    2132,
    2718,
    2030,
    1907,
    2109
  ],
  "464495": [
    984,
    984,
    969,
    1077,
    1109,
    1407,
    1000,
    1078,
    1058,
    984,
    1111,
    1171,
    984,
    1031,
    1015,
    999,
    1031,
    1031,
    1233,
    1546,
    1265,
    1140,
    1000,
    1202,
    1109
  ],
  "232247": [
    532,
    540,
    529,
    719,
    625,
    562,
    563,
    593,
    610,
    624,
    609,
    624,
    609,
    624,
    604,
    566,
    609,
    546,
    609,
    656,
    625,
    640,
    639,
    531,
    563
  ],
  "116123": [
    249,
    296,
    311,
    296,
    296,
    312,
    281,
    280,
    250,
    249,
    265,
    312,
    265,
    265,
    249,
    249,
    311,
    249,
    218,
    250,
    233,
    281,
    265,
    

In [69]:
https://paste.ubuntu.com/p/WQVBf7rDNw/

SyntaxError: invalid syntax (<ipython-input-69-84634ae2259f>, line 1)